<a href="https://colab.research.google.com/github/mrudulmamtani/Quant-Edge/blob/main/Advance_Price_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

def advanced_monte_carlo_option_price(S0, K, T, r, sigma, num_simulations, num_steps, option_type='call'):
    np.random.seed(42)
    dt = T / num_steps
    discount_factor = np.exp(-r * T)

    # Generate paths
    paths = np.zeros((num_simulations, num_steps + 1))
    paths[:, 0] = S0
    for t in range(1, num_steps + 1):
        Z = np.random.randn(num_simulations)
        paths[:, t] = paths[:, t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z)

    # Payoffs
    if option_type == 'call':
        payoffs = np.maximum(paths[:, -1] - K, 0)
    else:
        payoffs = np.maximum(K - paths[:, -1], 0)

    option_price = discount_factor * np.mean(payoffs)
    return option_price

# Parameters
S0 = 100  # Initial stock price
K = 100   # Strike price
T = 1     # Time to maturity (1 year)
r = 0.05  # Risk-free rate
sigma = 0.2  # Volatility
num_simulations = 10000
num_steps = 365  # Daily steps

mc_price = advanced_monte_carlo_option_price(S0, K, T, r, sigma, num_simulations, num_steps)
print(f"Advanced Monte Carlo Option Price: {mc_price}")


Advanced Monte Carlo Option Price: 10.302754123392171


In [2]:
def crr_binomial_option_price(S0, K, T, r, sigma, steps, option_type='call'):
    dt = T / steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    option_values = np.zeros(steps + 1)
    for i in range(steps + 1):
        S = S0 * (u ** (steps - i)) * (d ** i)
        option_values[i] = max(0, S - K) if option_type == 'call' else max(0, K - S)

    for step in range(steps - 1, -1, -1):
        for i in range(step + 1):
            option_values[i] = np.exp(-r * dt) * (p * option_values[i] + (1 - p) * option_values[i + 1])

    return option_values[0]

# Parameters
steps = 1000

binomial_price = crr_binomial_option_price(S0, K, T, r, sigma, steps)
print(f"CRR Binomial Option Price: {binomial_price}")


CRR Binomial Option Price: 10.448584103764572


In [3]:
from scipy.stats import norm

def black_scholes_with_greeks(S0, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        delta = norm.cdf(d1)
        gamma = norm.pdf(d1) / (S0 * sigma * np.sqrt(T))
        vega = S0 * norm.pdf(d1) * np.sqrt(T)
        theta = -(S0 * norm.pdf(d1) * sigma) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)
        rho = K * T * np.exp(-r * T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
        delta = -norm.cdf(-d1)
        gamma = norm.pdf(d1) / (S0 * sigma * np.sqrt(T))
        vega = S0 * norm.pdf(d1) * np.sqrt(T)
        theta = -(S0 * norm.pdf(d1) * sigma) / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2)
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2)

    greeks = {'Delta': delta, 'Gamma': gamma, 'Vega': vega, 'Theta': theta, 'Rho': rho}
    return price, greeks

bs_price, bs_greeks = black_scholes_with_greeks(S0, K, T, r, sigma)
print(f"Black-Scholes Option Price: {bs_price}")
print(f"Black-Scholes Greeks: {bs_greeks}")


Black-Scholes Option Price: 10.450583572185565
Black-Scholes Greeks: {'Delta': 0.6368306511756191, 'Gamma': 0.018762017345846895, 'Vega': 37.52403469169379, 'Theta': -6.414027546438197, 'Rho': 53.232481545376345}


In [5]:
def merged_option_price(mc_price, binomial_price, bs_price, weights):
    weighted_price = weights[0] * mc_price + weights[1] * binomial_price + weights[2] * bs_price
    return weighted_price

# Weights for each model
weights = [0.3, 0.3, 0.4]

merged_price = merged_option_price(mc_price, binomial_price, bs_price, weights)
print(f"Merged Option Price: {merged_price}")


Merged Option Price: 10.40563489702125
